In [31]:
# pip install google-generativeai
# pip install pymysql
# pip install langchain_experimental
# pip install sentence-transformers
# pip install chromadb

### Importing GooglePalm Model

In [11]:
api_key = ''

In [13]:
from langchain.llms import GooglePalm

llm = GooglePalm(google_api_key = api_key , temperature = 0.2)

In [14]:
poem  =  llm ('Write a poem on love for burger')
print(poem)

C:\Users\LENOVO\Desktop\New Python Env\NEW-VENV\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


**Ode to a Burger**

Oh, burger, how I love thee!
Let me count the ways.

I love your golden brown bun,
Your juicy patty,
Your melted cheese,
Your crisp lettuce,
Your juicy tomatoes,
And your tangy pickles.

I love the way you smell,
So fresh and inviting.
I love the way you taste,
So delicious and satisfying.
I love the way you make me feel,
So happy and content.

Oh, burger, you are the perfect food.
You are my one true love.

I will never stop loving you.

I will cherish you forever.

You are my everything.


### Connecting with Database

In [20]:
from langchain.utilities import SQLDatabase


db_user = "root"
db_password = ""
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)

print(db.table_info)

# URI is like URl it specifies what is your database what is your host usename password and so on


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

Database connected successfully

### Creating SQL Database chain


In [21]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm = llm ,db = db , verbose = True)   # It takes llm  and database

qns1 = db_chain("How many t-shirts do we have left for nike in extra small size and white color")

print(qns1)

C:\Users\LENOVO\Desktop\New Python Env\NEW-VENV\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(16,)]
Answer:16
> Finished chain.
{'query': 'How many t-shirts do we have left for nike in extra small size and white color', 'result': '16'}


Correct Answer

In [22]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")

C:\Users\LENOVO\Desktop\New Python Env\NEW-VENV\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('334'),)]
Answer:334
> Finished chain.


Wrong Answer :
It's giving  only 1 unit price but we need price of all inventory

Query should be :
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'


In [23]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('18639'),)]
Answer:18639
> Finished chain.


In [24]:
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(price * (1 - pct_discount)) 
FROM discounts 
JOIN t_shirts ON t_shirts.t_shirt_id = discounts.t_shirt_id 
WHERE brand = 'Levi' AND CURDATE() BETWEEN start_date AND end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'start_date' in 'where clause'")
[SQL: SELECT SUM(price * (1 - pct_discount)) 
FROM discounts 
JOIN t_shirts ON t_shirts.t_shirt_id = discounts.t_shirt_id 
WHERE brand = 'Levi' AND CURDATE() BETWEEN start_date AND end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Wrong Answer

In [25]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('12450.400000'),)]
Answer:12450.4
> Finished chain.


In [26]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('13395'),)]
Answer:13395
> Finished chain.


In [27]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(36,), (29,), (63,), (90,)]
Answer:36
> Finished chain.


In [28]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('218'),)]
Answer:218
> Finished chain.


In [50]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "16"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "18639"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
     (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
     group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
     """,
     'SQLResult': "Result of the SQL query",
     'Answer': "12450.4"} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : "13395"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "218"
     }
]

###  Creating Semantic Similarity Based example selector


create embedding on the few_shots

Store the embeddings in Chroma DB

Retrieve the the top most Semantically close example from the vector store

In [30]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2') 

In [52]:
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [57]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_texts(to_vectorize, embedding = embeddings, metadatas=few_shots)

In [61]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore = vectorstore,
    k=2)     # K =2 pulls 2 similar examples

In [62]:
example_selector.select_examples({"Question": "How many Adidas T shirts I have left inn my store?"})

[{'Answer': '16',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '218',
  'Question': "How many white color Levi's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'}]

### Providing Promt to LLM


In [63]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [64]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [65]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [66]:
#Query
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [70]:
# Creating few_shots prompt template

from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # connect llm with our vectore store if it is confused then take help from it
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [72]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)
# Adding prompt as an argument

In [73]:
new_chain("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('218'),)]
Answer:218
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '218'}